In [126]:
!mkdir -p raw processed output

In [127]:
import pandas as pd

In [128]:
df = pd.read_csv("raw/retail_sales_dataset.csv")

In [129]:
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [130]:
df.isna().sum()

,0
Transaction ID,0
Date,0
Customer ID,0
Gender,0
Age,0
Product Category,0
Quantity,0
Price per Unit,0
Total Amount,0


In [131]:
df.duplicated().sum()

np.int64(0)

In [132]:
df = df.drop_duplicates()

In [133]:
df.to_csv("processed/retail_sales_cleaned.csv", index=False)

In [134]:
df = pd.read_csv("processed/retail_sales_cleaned.csv")

In [135]:
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [136]:
import re

def standardize_columns(cols):
    return (
        cols.str.lower()
            .str.strip()
            .str.replace(r"[^\w\s]", "", regex=True)
            .str.replace(r"\s+", "_", regex=True)
    )

df.columns = standardize_columns(df.columns)
df.columns


Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount'],
      dtype='object')

In [137]:
numeric_candidates = [
    c for c in df.columns
    if any(x in c for x in ["sales", "revenue", "price", "amount", "quantity", "units", "profit"])
]

for col in numeric_candidates:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [138]:
cat_cols = df.select_dtypes(include="object").columns
df[cat_cols] = df[cat_cols].astype("category")

In [139]:
date_cols = [c for c in df.columns if "date" in c]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    1000 non-null   int64         
 1   date              1000 non-null   datetime64[ns]
 2   customer_id       1000 non-null   category      
 3   gender            1000 non-null   category      
 4   age               1000 non-null   int64         
 5   product_category  1000 non-null   category      
 6   quantity          1000 non-null   int64         
 7   price_per_unit    1000 non-null   int64         
 8   total_amount      1000 non-null   int64         
dtypes: category(3), datetime64[ns](1), int64(5)
memory usage: 91.3 KB


In [141]:
df.isna().sum()

,0
transaction_id,0
date,0
customer_id,0
gender,0
age,0
product_category,0
quantity,0
price_per_unit,0
total_amount,0


In [142]:
df.to_csv("processed/retail_sales_standardized.csv", index=False)

In [143]:
df = pd.read_csv("processed/retail_sales_standardized.csv")
df.head()

,transaction_id,date,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [144]:
if "profit" in df.columns:
    df["margin"] = df["profit"]
elif {"revenue", "cost"}.issubset(df.columns):
    df["margin"] = df["revenue"] - df["cost"]
else:
    df["margin"] = df["total_amount"] - (df["price_per_unit"] * df["quantity"])

In [145]:
df["margin_pct"] = df["margin"] / df["total_amount"]

In [146]:
high_value_threshold = df["total_amount"].quantile(0.75)
df["is_high_value"] = (df["total_amount"] >= high_value_threshold).astype(int)

In [147]:
high_margin_threshold = df["margin_pct"].quantile(0.75)
df["is_high_margin"] = (df["margin_pct"] >= high_margin_threshold).astype(int)

In [148]:
df["is_low_margin"] = (df["margin_pct"] <= 0.10).astype(int)

In [149]:
if "quantity" in df.columns:
    bulk_threshold = df["quantity"].quantile(0.75)
    df["is_bulk_purchase"] = (df["quantity"] >= bulk_threshold).astype(int)

In [150]:
if "customer_id" in df.columns:
    df["is_repeat_customer"] = (
        df.groupby("customer_id")["customer_id"].transform("count") > 1
    ).astype(int)

In [151]:
df[
    ["total_amount", "margin", "margin_pct", "is_high_value", "is_high_margin"]
].head()

,total_amount,margin,margin_pct,is_high_value,is_high_margin
0,150,0,0.0,0,1
1,1000,0,0.0,1,1
2,30,0,0.0,0,1
3,500,0,0.0,0,1
4,100,0,0.0,0,1


In [152]:
df.to_csv("processed/retail_sales_features.csv", index=False)

In [153]:
df = pd.read_csv("processed/retail_sales_features.csv")
df.head()

,transaction_id,date,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount,margin,margin_pct,is_high_value,is_high_margin,is_low_margin,is_bulk_purchase,is_repeat_customer
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,0,0.0,0,1,1,0,0
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,0,0.0,1,1,1,0,0
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,0,0.0,0,1,1,0,0
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,0,0.0,0,1,1,0,0
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,0,0.0,0,1,1,0,0


In [154]:
customer_cols = [
    c for c in df.columns
    if c.startswith("customer") or c in ["region", "segment", "is_repeat_customer"]
]

customers = (
    df[customer_cols]
    .drop_duplicates()
    .reset_index(drop=True)
)

customers.head()

,customer_id,is_repeat_customer
0,CUST001,0
1,CUST002,0
2,CUST003,0
3,CUST004,0
4,CUST005,0


In [155]:
product_cols = [
    c for c in df.columns
    if c.startswith("product") or c in ["category", "sub_category", "is_high_margin"]
]

products = (
    df[product_cols]
    .drop_duplicates()
    .reset_index(drop=True)
)

products.head()

,product_category,is_high_margin
0,Beauty,1
1,Clothing,1
2,Electronics,1


In [156]:
exclude_cols = set(customer_cols + product_cols)

orders = df[
    [c for c in df.columns if c not in exclude_cols]
].reset_index(drop=True)

orders.head()

,transaction_id,date,gender,age,quantity,price_per_unit,total_amount,margin,margin_pct,is_high_value,is_low_margin,is_bulk_purchase
0,1,2023-11-24,Male,34,3,50,150,0,0.0,0,1,0
1,2,2023-02-27,Female,26,2,500,1000,0,0.0,1,1,0
2,3,2023-01-13,Male,50,1,30,30,0,0.0,0,1,0
3,4,2023-05-21,Male,37,1,500,500,0,0.0,0,1,0
4,5,2023-05-06,Male,30,2,50,100,0,0.0,0,1,0


In [157]:
print("Customers:", customers.shape)
print("Products:", products.shape)
print("Orders:", orders.shape)

Customers: (1000, 2)
Products: (3, 2)
Orders: (1000, 12)


In [158]:
customers.to_csv("output/customers.csv", index=False)
products.to_csv("output/products.csv", index=False)
orders.to_csv("output/orders.csv", index=False)

In [159]:
customers = pd.read_csv("output/customers.csv")
products = pd.read_csv("output/products.csv")
orders = pd.read_csv("output/orders.csv")

In [160]:
customers.head()

,customer_id,is_repeat_customer
0,CUST001,0
1,CUST002,0
2,CUST003,0
3,CUST004,0
4,CUST005,0


In [161]:
products.head()

,product_category,is_high_margin
0,Beauty,1
1,Clothing,1
2,Electronics,1


In [162]:
orders.head()

,transaction_id,date,gender,age,quantity,price_per_unit,total_amount,margin,margin_pct,is_high_value,is_low_margin,is_bulk_purchase
0,1,2023-11-24,Male,34,3,50,150,0,0.0,0,1,0
1,2,2023-02-27,Female,26,2,500,1000,0,0.0,1,1,0
2,3,2023-01-13,Male,50,1,30,30,0,0.0,0,1,0
3,4,2023-05-21,Male,37,1,500,500,0,0.0,0,1,0
4,5,2023-05-06,Male,30,2,50,100,0,0.0,0,1,0


In [163]:
!ls output

customers.csv  orders.csv  products.csv
